## Logistic regression model
### reference
- [demo](https://github.com/torch/demos/blob/master/logistic-regression/example-logistic-regression.lua)
- [coursera lecture](https://github.com/didw/lecture/blob/master/machine-learning/machine-learning-ex2/ex2/ex2.m)

In [35]:
require 'torch'
require 'nn'
require 'optim'
require 'csvigo'

In [36]:
loaded = csvigo.load{path='ex2data1.txt', mode='raw'}

<csv>	parsing file: ex2data1.txt	
<csv>	parsing done	


In [37]:
data = torch.Tensor{ loaded }[1]:t()[{{1,2}}]:t()

In [38]:
label = torch.Tensor{ loaded }[1]:t()[3] + 1

In [39]:
mean = {}
stdv = {}
for i=1,2 do
    mean[i] = data[{{},{i}}]:mean()
    stdv[i] = data[{{},{i}}]:std()
    data[{{},{i}}]:add(-mean[i])
    data[{{},{i}}]:div(stdv[i])
end

### Define the model

In [40]:
linLayer = nn.Linear(2,2)
softMaxLayer = nn.LogSoftMax()
model = nn.Sequential()
model:add(linLayer)
model:add(nn.Sigmoid())
model:add(softMaxLayer)

### Define a loss function

In [41]:
criterion = nn.ClassNLLCriterion()

### Train the model (using SGD)

In [42]:
x, dl_dx = model:getParameters()

In [43]:
feval = function(x_new)
   if x ~= x_new then
      x:copy(x_new)
   end

   _nidx_ = (_nidx_ or 0) + 1
   if _nidx_ > (#data)[1] then _nidx_ = 1 end

   local inputs = data[_nidx_]
   local target = label[_nidx_]

   dl_dx:zero()

   local loss_x = criterion:forward(model:forward(inputs), target)
   model:backward(inputs, criterion:backward(model.output, target))

   return loss_x, dl_dx
end

In [44]:
sgd_params = {
   learningRate = 1e-3,
   learningRateDecay = 1e-4,
   weightDecay = 0,
   momentum = 0
}

In [45]:
epochs = 1e3

In [46]:
print('')
print('============================================================')
print('Training with SGD')
print('')

In [47]:
for i = 1,epochs do
   current_loss = 0
   for i = 1,(#data)[1] do
      _,fs = optim.sgd(feval,x,sgd_params)
      current_loss = current_loss + fs[1]
   end

   current_loss = current_loss / (#data)[1]
   if i % 100 == 0 then
        print('epoch = ' .. i .. 
         ' of ' .. epochs .. 
         ' current loss = ' .. current_loss)
    end
end

epoch = 100 of 1000 current loss = 0.63530930608779	


epoch = 200 of 1000 current loss = 0.59215649973138	


epoch = 300 of 1000 current loss = 0.56912134428907	


epoch = 400 of 1000 current loss = 0.55465762979904	


epoch = 500 of 1000 current loss = 0.54461204108006	


epoch = 600 of 1000 current loss = 0.53715093867064	


epoch = 700 of 1000 current loss = 0.53134109389978	


epoch = 800 of 1000 current loss = 0.5266563992714	


epoch = 900 of 1000 current loss = 0.52277671061168	


### Test the trained model

In [48]:
function maxIndex(a,b)
    if a>=b then return 1
    else return 2 end
end

In [49]:
function predictOut(a, b)
    local input = torch.Tensor(2)
    input[1] = a
    input[2] = b
    local logProbs = model:forward(input)
    local probs = torch.exp(logProbs)
    local prob1, prob2 = probs[1], probs[2]
    return maxIndex(prob1, prob2), prob1, prob2
end

In [50]:
corr = 0
for i = 1, (#data)[1] do
    local prediction = predictOut(data[i][1], data[i][2])
    
    -- print(prediction, label[i])
    if prediction == label[i] then
        corr = corr + 1
    end
end
print (corr / (#data)[1])

0.89	
